# Resnet imp

https://www.coursera.org/learn/custom-models-layers-loss-functions-with-tensorflow/lecture/YI2hH/resnet-code-walkthrough

https://www.youtube.com/watch?v=RQ4sMZiciuI

https://github.com/ANKITPODDER2000/Tensorflow-Advance/blob/main/Custom_Models_Layers_and_Loss_Functions_with_TensorFlow/Week4/resnet_implement.ipynb

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Layer, Activation,MaxPool2D
from tensorflow.keras.layers import BatchNormalization, Add, Dense
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D 
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
print("Version of tensorflow is : ",tf.__version__)

Version of tensorflow is :  2.6.0


In [2]:
class IdentityBlock(Model):
    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__(name='identityBlock')
        
        self.conv1 = Conv2D(filters, kernel_size, padding='same')
        self.batch_norm1 = BatchNormalization()
        
        self.conv2 = Conv2D(filters, kernel_size, padding='same')
        self.batch_norm2 = BatchNormalization()
        
        self.act = Activation('relu')
        self.add = Add()
    
    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.batch_norm1(x)
        x = self.act(x)
        
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.act(x)
        
        # the skip happens here,
        # we r adding the input to the deep features extracted by the deep learning layers
        x = self.add([x, input_tensor]) 
        x = self.act(x)
        
        return x
        

In [3]:
class ResNet(Model):
    def __init__(self, num_categories):
        super(ResNet, self).__init__()
        self.conv = Conv2D(64, 7, padding='same')
        self.batch_norm = BatchNormalization()
        self.act = Activation('relu')
        self.max_pool = MaxPool2D((3,3))
        
        self.identity_block1 = IdentityBlock(64, 3)
        self.identity_block2 = IdentityBlock(64, 3)
        
        self.global_pool = GlobalAveragePooling2D()
        self.classifier = Dense(num_categories, activation='softmax')
        
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.batch_norm(x)
        x = self.act(x)
        x = self.max_pool(x)
        
        x = self.identity_block1(x)
        x = self.identity_block2(x)
        
        x = self.global_pool(x)
        
        return self.classifier(x)
        

        

In [7]:
def preprocess(features):
#     print(features.keys())
    return tf.cast(features['image'], tf.float32) / 255. , features['label']
resnet = ResNet(10)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dataset = tfds.load(name='mnist' , split=tfds.Split.TRAIN)
dataset = dataset.map(preprocess).batch(32)

dataset_test = tfds.load(name='mnist' , split=tfds.Split.TEST)
dataset_test = dataset_test.map(preprocess).batch(32)


In [11]:
history = resnet.fit(dataset , epochs=3)

Epoch 1/3
1875/1875 [==============================] - 16s 6ms/step - loss: 0.1405 - accuracy: 0.9645
Epoch 2/3
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0469 - accuracy: 0.9867
Epoch 3/3
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0345 - accuracy: 0.9900


In [12]:
resnet.evaluate(dataset_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.1119 - accuracy: 0.9637


[0.1119321659207344, 0.963699996471405]

In [13]:
resnet.summary()

Model: "res_net_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           multiple                  3200      
_________________________________________________________________
batch_normalization_15 (Batc multiple                  256       
_________________________________________________________________
activation_9 (Activation)    multiple                  0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 multiple                  0         
_________________________________________________________________
identityBlock (IdentityBlock multiple                  74368     
_________________________________________________________________
identityBlock (IdentityBlock multiple                  74368     
_________________________________________________________________
global_average_pooling2d_3 ( multiple                  0 